# Importing Data

In [1]:
library(QuantPsyc)
library(dplyr)
library(haven)
library(estimatr)
library(plm)
library(sandwich)
library(lmtest)

Loading required package: boot
Loading required package: MASS

Attaching package: ‘QuantPsyc’

The following object is masked from ‘package:base’:

    norm


Attaching package: ‘dplyr’

The following object is masked from ‘package:MASS’:

    select

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: Formula

Attaching package: ‘plm’

The following objects are masked from ‘package:dplyr’:

    between, lag, lead

Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric



# Loading the data sets
### Global Preferences Survey (GPS) - by country and by individual

In [2]:
country_level_GPS_data = read_dta("data/GPS_dataset_country_level/country_v11.dta")
individual_level_GPS_data = read_dta("data/GPS_dataset_individual_level/individual_v11.dta")

The data is an index among a number of preferences by country.

### Gallup World Poll data

In [3]:
#gallup_wp_data = read_dta("data/The_Gallup_World_Poll_032219_subset.dta")
gallup_wp_data2 = read_dta("data/The_Gallup_World_Poll_032219_subset2.dta")

## Merging the GPS and Gallup WP data by `wpid`

In [5]:
# rename the gallup id column to match the wpid
individual_level_GPS_data <- individual_level_GPS_data %>% rename(wpid = id_gallup)

In [6]:
merge_gallup_indGPS <- merge(individual_level_GPS_data,
                             gallup_wp_data2,
                             by = "wpid",
                            how = 'left')

## Checking data sparsity

In [7]:
na_count <- sapply(merge_gallup_indGPS, function(y) sum(length(which(is.na(y)))))
na_count <- data.frame(na_count)
na_count$name <- rownames(na_count)
na_count$na_count <- na_count$na_count / nrow(merge_gallup_indGPS) * 100
na_count = na_count[order(na_count),]
na_count#[order(na_count),]

,na_count,name
wpid,0,wpid
country,0,country
isocode,0,isocode
ison,0,ison
region,0,region
language,0,language
wgt.x,0,wgt.x
WP5889,0,WP5889
WP5,0,WP5
wave,0,wave


# Table V

In [49]:
table_v_data <- merge_gallup_indGPS

# drop where age is 100 (refused)
table_v_data <- table_v_data[!(table_v_data$WP1220==100),]

# divide age by 100 (per paper)
table_v_data$age <- table_v_data$WP1220 / 100

# add in age ^ 2
table_v_data$age_sq <- (table_v_data$WP1220 / 100) ^ 2

## Table V - Patience
Cleaning up data by dropping any rows with missing data.

### Country FE

In [56]:
table_v_data_patience_country = table_v_data[complete.cases(table_v_data$subj_math_skills, 
                                           table_v_data$patience, 
                                           table_v_data$country,
                                           #table_v_data$region,
                                          table_v_data$WP1220,
                                          table_v_data$WP1219), ]


# check how many rows were dropped
print("number of rows dropped: ")
nrow(merge_gallup_indGPS) - nrow(table_v_data_patience_country)
print("% of data: ")
(nrow(merge_gallup_indGPS) - nrow(table_v_data_patience_country))/nrow(merge_gallup_indGPS) * 100
print("number of rows remaining: ")
nrow(table_v_data_patience_country)


[1] "number of rows dropped: "


[1] 1836

[1] "% of data: "


[1] 2.285373

[1] "number of rows remaining: "


[1] 78501

In [65]:
table_v_patience_country_FE = lm(patience ~ subj_math_skills + 
                                                  age + 
                                                  age_sq +
                                                  WP1219 +
                                                  country, 
                    data = table_v_data_patience_country)

summary(table_v_patience_country_FE)


Call:
lm(formula = patience ~ subj_math_skills + age + age_sq + WP1219 + 
    country, data = table_v_data_patience_country)

Residuals:
    Min      1Q  Median      3Q     Max 
-2.4617 -0.6062 -0.1904  0.3898  3.7721 

Coefficients:
                             Estimate Std. Error t value Pr(>|t|)    
(Intercept)                 -0.313632   0.036403  -8.616  < 2e-16 ***
subj_math_skills             0.028346   0.001208  23.472  < 2e-16 ***
age                          0.716555   0.095405   7.511 5.95e-14 ***
age_sq                      -1.446457   0.102079 -14.170  < 2e-16 ***
WP1219                      -0.055682   0.006644  -8.381  < 2e-16 ***
countryAlgeria               0.264464   0.040775   6.486 8.87e-11 ***
countryArgentina             0.057135   0.041322   1.383 0.166770    
countryAustralia             0.997584   0.041465  24.058  < 2e-16 ***
countryAustria               0.844206   0.041558  20.314  < 2e-16 ***
countryBangladesh            0.307982   0.041526   7.417 1.21e-13

### Region FE

In [58]:
table_v_data_patience_region = table_v_data[complete.cases(table_v_data$subj_math_skills, 
                                           table_v_data$patience, 
                                           #table_v_data$country,
                                           table_v_data$region,
                                          table_v_data$WP1220,
                                          table_v_data$WP1219), ]


# check how many rows were dropped
print("number of rows dropped: ")
nrow(merge_gallup_indGPS) - nrow(table_v_data_patience_region)
print("% of data: ")
(nrow(merge_gallup_indGPS) - nrow(table_v_data_patience_region))/nrow(merge_gallup_indGPS) * 100
print("number of rows remaining: ")
nrow(table_v_data_patience_region)


[1] "number of rows dropped: "


[1] 1836

[1] "% of data: "


[1] 2.285373

[1] "number of rows remaining: "


[1] 78501

In [64]:
table_v_patience_region_FE = lm(patience ~ subj_math_skills + 
                                                  age + 
                                                  age_sq +
                                                  WP1219 +
                                                  region, 
                    data = table_v_data_patience_region)

summary(table_v_patience_region_FE)


Call:
lm(formula = patience ~ subj_math_skills + age + age_sq + WP1219 + 
    region, data = table_v_data_patience_region)

Residuals:
    Min      1Q  Median      3Q     Max 
-2.6240 -0.5814 -0.1630  0.3891  3.4502 

Coefficients:
                                                         Estimate Std. Error
(Intercept)                                            -0.1527559  0.0386677
subj_math_skills                                        0.0258907  0.0012181
age                                                     0.7547257  0.0944584
age_sq                                                 -1.4877604  0.1011103
WP1219                                                 -0.0591602  0.0065690
region El-Garbia                                       -0.6302237  0.1016456
region Khon Kaen                                        0.0087556  0.1663094
region Rajasthan                                       -0.2840289  0.0792903
region Republic of Altay                               -0.2610757  0.11266

## Risk Taking

Clean up the data in the same manner.

### Country FE

In [61]:
table_v_data_risktaking_country_FE = table_v_data[complete.cases(table_v_data$subj_math_skills, 
                                           table_v_data$risktaking, 
                                           table_v_data$country,
                                           #table_v_data$region,
                                          table_v_data$WP1220,
                                          table_v_data$WP1219), ]

# check how many rows were dropped
print("number of rows dropped: ")
nrow(merge_gallup_indGPS) - nrow(table_v_data_risktaking_country_FE)
print("% of data: ")
(nrow(merge_gallup_indGPS) - nrow(table_v_data_risktaking_country_FE))/nrow(merge_gallup_indGPS) * 100
print("number of rows remaining: ")
nrow(table_v_data_risktaking_country_FE)


[1] "number of rows dropped: "


[1] 1892

[1] "% of data: "


[1] 2.355079

[1] "number of rows remaining: "


[1] 78445

In [62]:
table_v_risktaking_country_FE = lm(risktaking ~ subj_math_skills + 
                                                  age + 
                                                  age_sq +
                                                  WP1219 +
                                                  country, 
                    data = table_v_data_risktaking_country_FE)

summary(table_v_risktaking_country_FE)


Call:
lm(formula = risktaking ~ subj_math_skills + age + age_sq + WP1219 + 
    country, data = table_v_data_patience)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.0250 -0.6321 -0.0441  0.6025  3.4221 

Coefficients:
                             Estimate Std. Error t value Pr(>|t|)    
(Intercept)                  0.381224   0.036379  10.479  < 2e-16 ***
subj_math_skills             0.045960   0.001208  38.033  < 2e-16 ***
age                         -0.081867   0.095610  -0.856 0.391860    
age_sq                      -1.197682   0.102419 -11.694  < 2e-16 ***
WP1219                      -0.174133   0.006642 -26.216  < 2e-16 ***
countryAlgeria               0.240962   0.040689   5.922 3.19e-09 ***
countryArgentina            -0.001195   0.041330  -0.029 0.976936    
countryAustralia             0.086186   0.041390   2.082 0.037318 *  
countryAustria              -0.120437   0.041472  -2.904 0.003685 ** 
countryBangladesh           -0.288778   0.041461  -6.965 3.31e-12 ***
c

### Region FE

In [63]:
table_v_data_risktaking_region_FE = table_v_data[complete.cases(table_v_data$subj_math_skills, 
                                           table_v_data$risktaking, 
                                           #table_v_data$country,
                                           table_v_data$region,
                                          table_v_data$WP1220,
                                          table_v_data$WP1219), ]


# check how many rows were dropped
print("number of rows dropped: ")
nrow(merge_gallup_indGPS) - nrow(table_v_data_risktaking_region_FE)
print("% of data: ")
(nrow(merge_gallup_indGPS) - nrow(table_v_data_risktaking_region_FE))/nrow(merge_gallup_indGPS) * 100
print("number of rows remaining: ")
nrow(table_v_data_risktaking_region_FE)


[1] "number of rows dropped: "


[1] 1892

[1] "% of data: "


[1] 2.355079

[1] "number of rows remaining: "


[1] 78445

In [66]:
table_v_risktaking_region_FE = lm(risktaking ~ subj_math_skills + 
                                                  age + 
                                                  age_sq +
                                                  WP1219 +
                                                  region, 
                    data = table_v_data_risktaking_region_FE)

summary(table_v_risktaking_region_FE)


Call:
lm(formula = risktaking ~ subj_math_skills + age + age_sq + WP1219 + 
    region, data = table_v_data_risktaking_region_FE)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.0347 -0.6130 -0.0418  0.5795  4.0157 

Coefficients:
                                                         Estimate Std. Error
(Intercept)                                             0.3783011  0.0386518
subj_math_skills                                        0.0423407  0.0012114
age                                                    -0.0859339  0.0940243
age_sq                                                 -1.1841627  0.1006773
WP1219                                                 -0.1766539  0.0065332
region El-Garbia                                       -0.6647554  0.1011167
region Khon Kaen                                       -0.2674797  0.1653763
region Rajasthan                                       -0.5705889  0.0793690
region Republic of Altay                               -0.6004746  

## Positive Reciprocity

### Country FE

In [71]:
table_v_data_posrecip_country_FE = table_v_data[complete.cases(table_v_data$subj_math_skills, 
                                           table_v_data$posrecip, 
                                           table_v_data$country,
                                           #table_v_data$region,
                                          table_v_data$WP1220,
                                          table_v_data$WP1219), ]

# check how many rows were dropped
print("number of rows dropped: ")
nrow(merge_gallup_indGPS) - nrow(table_v_data_posrecip_country_FE)
print("% of data: ")
(nrow(merge_gallup_indGPS) - nrow(table_v_data_posrecip_country_FE))/nrow(merge_gallup_indGPS) * 100
print("number of rows remaining: ")
nrow(table_v_data_posrecip_country_FE)


[1] "number of rows dropped: "


[1] 1468

[1] "% of data: "


[1] 1.827302

[1] "number of rows remaining: "


[1] 78869

In [73]:
table_v_posrecip_country_FE = lm(posrecip ~ subj_math_skills + 
                                                  age + 
                                                  age_sq +
                                                  WP1219 +
                                                  country, 
                    data = table_v_data_posrecip_country_FE)

summary(table_v_posrecip_country_FE)


Call:
lm(formula = posrecip ~ subj_math_skills + age + age_sq + WP1219 + 
    country, data = table_v_data_posrecip_country_FE)

Residuals:
    Min      1Q  Median      3Q     Max 
-4.5130 -0.5818  0.1270  0.6915  2.6903 

Coefficients:
                             Estimate Std. Error t value Pr(>|t|)    
(Intercept)                 -0.127974   0.037091  -3.450 0.000560 ***
subj_math_skills             0.038205   0.001229  31.078  < 2e-16 ***
age                          1.022318   0.097059  10.533  < 2e-16 ***
age_sq                      -1.170374   0.103741 -11.282  < 2e-16 ***
WP1219                       0.048966   0.006770   7.233 4.78e-13 ***
countryAlgeria              -0.889318   0.041597 -21.380  < 2e-16 ***
countryArgentina            -0.122996   0.042091  -2.922 0.003478 ** 
countryAustralia            -0.291573   0.042250  -6.901 5.20e-12 ***
countryAustria              -0.169165   0.042237  -4.005 6.20e-05 ***
countryBangladesh           -0.147207   0.042331  -3.478 0.000

### Region FE

In [72]:
table_v_data_posrecip_region_FE = table_v_data[complete.cases(table_v_data$subj_math_skills, 
                                           table_v_data$posrecip, 
                                           #table_v_data$country,
                                           table_v_data$region,
                                          table_v_data$WP1220,
                                          table_v_data$WP1219), ]

# check how many rows were dropped
print("number of rows dropped: ")
nrow(merge_gallup_indGPS) - nrow(table_v_data_posrecip_region_FE)
print("% of data: ")
(nrow(merge_gallup_indGPS) - nrow(table_v_data_posrecip_region_FE))/nrow(merge_gallup_indGPS) * 100
print("number of rows remaining: ")
nrow(table_v_data_posrecip_region_FE)


[1] "number of rows dropped: "


[1] 1468

[1] "% of data: "


[1] 1.827302

[1] "number of rows remaining: "


[1] 78869

In [74]:
table_v_posrecip_region_FE = lm(posrecip ~ subj_math_skills + 
                                                  age + 
                                                  age_sq +
                                                  WP1219 +
                                                  region, 
                    data = table_v_data_posrecip_region_FE)

summary(table_v_posrecip_region_FE)


Call:
lm(formula = posrecip ~ subj_math_skills + age + age_sq + WP1219 + 
    region, data = table_v_data_posrecip_region_FE)

Residuals:
    Min      1Q  Median      3Q     Max 
-4.4944 -0.5436  0.1061  0.6362  3.9487 

Coefficients:
                                                         Estimate Std. Error
(Intercept)                                            -0.3463168  0.0384672
subj_math_skills                                        0.0382179  0.0012105
age                                                     1.0671184  0.0938245
age_sq                                                 -1.2367853  0.1003292
WP1219                                                  0.0545276  0.0065354
region El-Garbia                                        0.6300237  0.1013216
region Khon Kaen                                        0.4315069  0.1657985
region Rajasthan                                        0.0795457  0.0797191
region Republic of Altay                               -0.0170009  0.11

## Neg. Reciprocity

### Country FE

In [76]:
table_v_data_negrecip_country_FE = table_v_data[complete.cases(table_v_data$subj_math_skills, 
                                           table_v_data$negrecip, 
                                           table_v_data$country,
                                           #table_v_data$region,
                                          table_v_data$WP1220,
                                          table_v_data$WP1219), ]

# check how many rows were dropped
print("number of rows dropped: ")
nrow(merge_gallup_indGPS) - nrow(table_v_data_negrecip_country_FE)
print("% of data: ")
(nrow(merge_gallup_indGPS) - nrow(table_v_data_negrecip_country_FE))/nrow(merge_gallup_indGPS) * 100
print("number of rows remaining: ")
nrow(table_v_data_negrecip_country_FE)


[1] "number of rows dropped: "


[1] 2816

[1] "% of data: "


[1] 3.505234

[1] "number of rows remaining: "


[1] 77521

In [85]:
table_v_negrecip_country_FE = lm(negrecip ~ subj_math_skills + 
                                                  age + 
                                                  age_sq +
                                                  WP1219 +
                                                  country, 
                    data = table_v_data_negrecip_country_FE)

summary(table_v_negrecip_country_FE)


Call:
lm(formula = negrecip ~ subj_math_skills + age + age_sq + WP1219 + 
    country, data = table_v_data_negrecip_country_FE)

Residuals:
    Min      1Q  Median      3Q     Max 
-2.4359 -0.7161 -0.0138  0.6409  3.2614 

Coefficients:
                             Estimate Std. Error t value Pr(>|t|)    
(Intercept)                  0.502975   0.037700  13.341  < 2e-16 ***
subj_math_skills             0.040092   0.001253  31.989  < 2e-16 ***
age                         -0.356355   0.098964  -3.601 0.000317 ***
age_sq                      -0.448261   0.105937  -4.231 2.33e-05 ***
WP1219                      -0.129949   0.006886 -18.873  < 2e-16 ***
countryAlgeria              -0.111512   0.042132  -2.647 0.008129 ** 
countryArgentina            -0.433882   0.042866 -10.122  < 2e-16 ***
countryAustralia            -0.229792   0.042910  -5.355 8.57e-08 ***
countryAustria              -0.343933   0.043107  -7.979 1.50e-15 ***
countryBangladesh           -0.161253   0.042965  -3.753 0.000

### Region FE

In [77]:
table_v_data_negrecip_region_FE = table_v_data[complete.cases(table_v_data$subj_math_skills, 
                                           table_v_data$negrecip, 
                                           #table_v_data$country,
                                           table_v_data$region,
                                          table_v_data$WP1220,
                                          table_v_data$WP1219), ]

# check how many rows were dropped
print("number of rows dropped: ")
nrow(merge_gallup_indGPS) - nrow(table_v_data_negrecip_region_FE)
print("% of data: ")
(nrow(merge_gallup_indGPS) - nrow(table_v_data_negrecip_region_FE))/nrow(merge_gallup_indGPS) * 100
print("number of rows remaining: ")
nrow(table_v_data_negrecip_region_FE)


[1] "number of rows dropped: "


[1] 2816

[1] "% of data: "


[1] 3.505234

[1] "number of rows remaining: "


[1] 77521

In [84]:
table_v_negrecip_region_FE = lm(negrecip ~ subj_math_skills + 
                                                  age + 
                                                  age_sq +
                                                  WP1219 +
                                                  region, 
                    data = table_v_data_negrecip_region_FE)

summary(table_v_negrecip_region_FE)


Call:
lm(formula = negrecip ~ subj_math_skills + age + age_sq + WP1219 + 
    region, data = table_v_data_negrecip_region_FE)

Residuals:
    Min      1Q  Median      3Q     Max 
-2.6332 -0.6628 -0.0198  0.6003  3.3313 

Coefficients:
                                                         Estimate Std. Error
(Intercept)                                             9.799e-01  4.090e-02
subj_math_skills                                        3.537e-02  1.247e-03
age                                                    -3.544e-01  9.665e-02
age_sq                                                 -4.348e-01  1.035e-01
WP1219                                                 -1.348e-01  6.716e-03
region El-Garbia                                       -1.074e+00  1.039e-01
region Khon Kaen                                       -4.327e-01  1.693e-01
region Rajasthan                                       -1.786e-01  8.376e-02
region Republic of Altay                               -8.496e-01  1.15

## Altruism

### Country FE

In [89]:
table_v_data_altruism_country_FE = table_v_data[complete.cases(table_v_data$subj_math_skills, 
                                           table_v_data$altruism, 
                                           table_v_data$country,
                                           #table_v_data$region,
                                          table_v_data$WP1220,
                                          table_v_data$WP1219), ]

# check how many rows were dropped
print("number of rows dropped: ")
nrow(merge_gallup_indGPS) - nrow(table_v_data_altruism_country_FE)
print("% of data: ")
(nrow(merge_gallup_indGPS) - nrow(table_v_data_altruism_country_FE))/nrow(merge_gallup_indGPS) * 100
print("number of rows remaining: ")
nrow(table_v_data_altruism_country_FE)


[1] "number of rows dropped: "


[1] 1705

[1] "% of data: "


[1] 2.12231

[1] "number of rows remaining: "


[1] 78632

In [90]:
table_v_altruism_country_FE = lm(altruism ~ subj_math_skills + 
                                                  age + 
                                                  age_sq +
                                                  WP1219 +
                                                  country, 
                    data = table_v_data_altruism_country_FE)

summary(table_v_altruism_country_FE)


Call:
lm(formula = altruism ~ subj_math_skills + age + age_sq + WP1219 + 
    country, data = table_v_data_altruism_country_FE)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.4089 -0.5904  0.0508  0.6322  3.4246 

Coefficients:
                             Estimate Std. Error t value Pr(>|t|)    
(Intercept)                 -0.164715   0.036937  -4.459 8.23e-06 ***
subj_math_skills             0.043666   0.001226  35.631  < 2e-16 ***
age                         -0.006094   0.096788  -0.063 0.949798    
age_sq                       0.014841   0.103468   0.143 0.885943    
WP1219                       0.100291   0.006747  14.864  < 2e-16 ***
countryAlgeria              -0.177565   0.041393  -4.290 1.79e-05 ***
countryArgentina            -0.180438   0.041896  -4.307 1.66e-05 ***
countryAustralia            -0.087440   0.042044  -2.080 0.037554 *  
countryAustria              -0.267882   0.042083  -6.366 1.96e-10 ***
countryBangladesh            0.735831   0.042134  17.464  < 2e

### Region FE

In [83]:
table_v_data_altruism_region_FE = table_v_data[complete.cases(table_v_data$subj_math_skills, 
                                           table_v_data$altruism, 
                                           #table_v_data$country,
                                           table_v_data$region,
                                          table_v_data$WP1220,
                                          table_v_data$WP1219), ]

# check how many rows were dropped
print("number of rows dropped: ")
nrow(merge_gallup_indGPS) - nrow(table_v_data_altruism_region_FE)
print("% of data: ")
(nrow(merge_gallup_indGPS) - nrow(table_v_data_altruism_region_FE))/nrow(merge_gallup_indGPS) * 100
print("number of rows remaining: ")
nrow(table_v_data_altruism_region_FE)


[1] "number of rows dropped: "


[1] 1705

[1] "% of data: "


[1] 2.12231

[1] "number of rows remaining: "


[1] 78632

In [91]:
table_v_altruism_region_FE = lm(altruism ~ subj_math_skills + 
                                                  age + 
                                                  age_sq +
                                                  WP1219 +
                                                  region, 
                    data = table_v_data_altruism_region_FE)

summary(table_v_altruism_region_FE)


Call:
lm(formula = altruism ~ subj_math_skills + age + age_sq + WP1219 + 
    region, data = table_v_data_altruism_region_FE)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.7505 -0.5544  0.0476  0.5969  3.5573 

Coefficients:
                                                         Estimate Std. Error
(Intercept)                                            -4.652e-01  3.916e-02
subj_math_skills                                        3.995e-02  1.230e-03
age                                                    -9.161e-03  9.533e-02
age_sq                                                  3.308e-03  1.020e-01
WP1219                                                  9.815e-02  6.637e-03
region El-Garbia                                        9.519e-01  1.028e-01
region Khon Kaen                                       -5.645e-01  1.682e-01
region Rajasthan                                       -3.391e-01  8.113e-02
region Republic of Altay                                3.067e-01  1.13

## Trust

In [93]:
table_v_data_trust_country_FE = table_v_data[complete.cases(table_v_data$subj_math_skills, 
                                           table_v_data$trust, 
                                           table_v_data$country,
                                           #table_v_data$region,
                                          table_v_data$WP1220,
                                          table_v_data$WP1219), ]

# check how many rows were dropped
print("number of rows dropped: ")
nrow(merge_gallup_indGPS) - nrow(table_v_data_trust_country_FE)
print("% of data: ")
(nrow(merge_gallup_indGPS) - nrow(table_v_data_trust_country_FE))/nrow(merge_gallup_indGPS) * 100
print("number of rows remaining: ")
nrow(table_v_data_trust_country_FE)


[1] "number of rows dropped: "


[1] 2523

[1] "% of data: "


[1] 3.140521

[1] "number of rows remaining: "


[1] 77814

In [94]:
table_v_trust_country_FE = lm(trust ~ subj_math_skills + 
                                                  age + 
                                                  age_sq +
                                                  WP1219 +
                                                  country, 
                    data = table_v_data_trust_country_FE)

summary(table_v_trust_country_FE)


Call:
lm(formula = trust ~ subj_math_skills + age + age_sq + WP1219 + 
    country, data = table_v_data_trust_country_FE)

Residuals:
     Min       1Q   Median       3Q      Max 
-2.94114 -0.59920  0.01988  0.65221  2.69613 

Coefficients:
                             Estimate Std. Error t value Pr(>|t|)    
(Intercept)                 -0.143477   0.037934  -3.782 0.000156 ***
subj_math_skills             0.056414   0.001253  45.036  < 2e-16 ***
age                          0.366333   0.098841   3.706 0.000210 ***
age_sq                       0.031573   0.105721   0.299 0.765213    
WP1219                       0.065555   0.006885   9.522  < 2e-16 ***
countryAlgeria              -0.543042   0.042385 -12.812  < 2e-16 ***
countryArgentina            -0.666484   0.043027 -15.490  < 2e-16 ***
countryAustralia            -0.132304   0.043101  -3.070 0.002144 ** 
countryAustria              -0.371316   0.043075  -8.620  < 2e-16 ***
countryBangladesh           -0.485851   0.043239 -11.236  

### Region FE

In [95]:
table_v_data_trust_region_FE = table_v_data[complete.cases(table_v_data$subj_math_skills, 
                                           table_v_data$trust, 
                                           #table_v_data$country,
                                           table_v_data$region,
                                          table_v_data$WP1220,
                                          table_v_data$WP1219), ]

# check how many rows were dropped
print("number of rows dropped: ")
nrow(merge_gallup_indGPS) - nrow(table_v_data_trust_region_FE)
print("% of data: ")
(nrow(merge_gallup_indGPS) - nrow(table_v_data_trust_region_FE))/nrow(merge_gallup_indGPS) * 100
print("number of rows remaining: ")
nrow(table_v_data_trust_region_FE)


[1] "number of rows dropped: "


[1] 2523

[1] "% of data: "


[1] 3.140521

[1] "number of rows remaining: "


[1] 77814

In [96]:
table_v_trust_region_FE = lm(trust ~ subj_math_skills + 
                                                  age + 
                                                  age_sq +
                                                  WP1219 +
                                                  region, 
                    data = table_v_data_trust_region_FE)

summary(table_v_trust_region_FE)


Call:
lm(formula = trust ~ subj_math_skills + age + age_sq + WP1219 + 
    region, data = table_v_data_trust_region_FE)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.1715 -0.5746  0.0145  0.6231  2.8801 

Coefficients:
                                                         Estimate Std. Error
(Intercept)                                            -0.4757896  0.0400769
subj_math_skills                                        0.0546173  0.0012567
age                                                     0.3218036  0.0973861
age_sq                                                  0.0725806  0.1042113
WP1219                                                  0.0590249  0.0067745
region El-Garbia                                        0.7066959  0.1044662
region Khon Kaen                                        0.8622022  0.1708032
region Rajasthan                                       -0.7982421  0.0859107
region Republic of Altay                               -0.1880438  0.1149856
